---<br>
jupyter:<br>
  jupytext:<br>
    text_representation:<br>
      extension: .py<br>
      format_name: light<br>
      format_version: '1.5'<br>
      jupytext_version: 1.14.5<br>
  kernelspec:<br>
    display_name: Python 3 (ipykernel)<br>
    language: python<br>
    name: python3<br>
---

# Stokes Benchmark SolCx<br>
<br>
<br>
options = PETSc.Options()<br>
options["help"] = None

%%

In [1]:
import petsc4py
from petsc4py import PETSc

In [2]:
import underworld3 as uw
from underworld3.systems import Stokes
from underworld3 import function
import numpy as np

%%

In [3]:
n_els = 16
mesh = uw.meshing.UnstructuredSimplexBox(
    minCoords=(0.0, 0.0), maxCoords=(1.0, 1.0), cellSize=1 / 20, qdegree=2
)

%%

In [4]:
v = uw.discretisation.MeshVariable("U", mesh, mesh.dim, degree=2)
p = uw.discretisation.MeshVariable("P", mesh, 1, degree=1)

%%

In [5]:
stokes = uw.systems.Stokes(mesh, velocityField=v, pressureField=p)
stokes.constitutive_model = uw.systems.constitutive_models.ViscousFlowModel(mesh.dim)
stokes.constitutive_model.Parameters.viscosity = 1

%%<br>
Set some things

In [6]:
import sympy
from sympy import Piecewise

In [7]:
mesh.dm.view()

DM Object: uw_mesh 1 MPI process
  type: plex
uw_mesh in 2 dimensions:
  Number of 0-cells per rank: 513
  Number of 1-cells per rank: 1456
  Number of 2-cells per rank: 944
Labels:
  depth: 3 strata with value/size (0 (513), 1 (1456), 2 (944))
  All_Boundaries: 1 strata with value/size (1001 (80))
  Bottom: 1 strata with value/size (11 (39))
  Elements: 1 strata with value/size (99999 (1377))
  Left: 1 strata with value/size (14 (39))
  Right: 1 strata with value/size (13 (39))
  Top: 1 strata with value/size (12 (39))
  celltype: 3 strata with value/size (0 (513), 1 (1456), 3 (944))
Field U:
  adjacency FEM
Field P:
  adjacency FEM


In [8]:
x,y = mesh.X

In [9]:
res = 1 / n_els
hw = 1000 / res
surface_fn = sympy.exp(-((y - 1.0) ** 2) * hw)
base_fn = sympy.exp(-(y**2) * hw)
right_fn = sympy.exp(-((x - 1.0) ** 2) * hw)
left_fn = sympy.exp(-(x**2) * hw)

In [10]:
eta_0 = 1.0
x_c = 0.5
f_0 = 1.0

In [11]:
stokes.penalty = 0.0
stokes.bodyforce = sympy.Matrix(
    [
        0,
        Piecewise(
            (f_0, x > x_c),
            (0.0, True),
        ),
    ]
)

+<br>
This is the other way to impose no vertical 

In [12]:
##stokes.bodyforce[0] -= 1.0e6 * v.sym[0] * (left_fn + right_fn)
##stokes.bodyforce[1] -= 1.0e6 * v.sym[1] * (surface_fn + base_fn)
# -

In [13]:
stokes.saddle_preconditioner = 1 / stokes.constitutive_model.Parameters.viscosity

+<br>
free slip.<br>
note with petsc we always need to provide a vector of correct cardinality.

In [14]:
stokes.add_dirichlet_bc(
    (0.0, 0.0), ["Top", "Bottom"], 1
)  # top/bottom: components, function, markers

In [15]:
stokes.add_dirichlet_bc(
    (0.0,), ["Left", "Right"], 0
)  # left/right: components, function, markers
# -

We may need to adjust the tolerance if $\Delta \eta$ is large

In [16]:
##stokes.petsc_options["snes_rtol"] = 1.0e-6
##stokes.petsc_options["ksp_rtol"] = 1.0e-6
##stokes.petsc_options["snes_max_it"] = 10

In [17]:
##stokes.petsc_options["snes_monitor"]= None
##stokes.petsc_options["ksp_monitor"] = None

%%<br>
Solve time

In [16]:
stokes.solve()

  0 SNES Function norm 0.0185536 
  1 SNES Function norm 5.88788e-08 
Nonlinear Stokes_1_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 1


3

### Visualise it !

+<br>
check the mesh if in a notebook / serial

In [19]:
import mpi4py

In [20]:
if mpi4py.MPI.COMM_WORLD.size == 1:
    import numpy as np
    import pyvista as pv
    import vtk
    
    pv.start_xvfb()
    pv.global_theme.background = "white"
    pv.global_theme.window_size = [750, 1200]
    pv.global_theme.antialiasing = True
    pv.global_theme.jupyter_backend = "panel"
    pv.global_theme.smooth_shading = True
    mesh.vtk("tmp_mesh.vtk")
    pvmesh = pv.read("tmp_mesh.vtk")
    pvmesh.point_data["P"] = uw.function.evaluate(p.sym[0], mesh.data)
    pvmesh.point_data["V"] = uw.function.evaluate(v.sym.dot(v.sym), mesh.data)
    arrow_loc = np.zeros((stokes.u.coords.shape[0], 3))
    arrow_loc[:, 0:2] = stokes.u.coords[...]
    arrow_length = np.zeros((stokes.u.coords.shape[0], 3))
    arrow_length[:, 0] = uw.function.evaluate(stokes.u.sym[0], stokes.u.coords)
    arrow_length[:, 1] = uw.function.evaluate(stokes.u.sym[1], stokes.u.coords)
    pl = pv.Plotter(window_size=[1000, 1000])
    pl.add_axes()
    pl.add_mesh(
        pvmesh,
        cmap="coolwarm",
        edge_color="Black",
        show_edges=True,
        scalars="V",
        use_transparency=False,
        opacity=1.0,
    )
    pl.add_arrows(arrow_loc, arrow_length, mag=3)
    pl.show(cpos="xy")
# -

/Users/maxwilliams/anaconda3/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2577: PyVistaDeprecationWarning: antialising is deprecated.  Please use `anti_aliasing` instead.
  warnings.warn(
/Users/maxwilliams/anaconda3/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2545: PyVistaDeprecationWarning: `anti_aliasing` is now a string or None and must be either "ssaa", "msaa", "fxaa", or None
  warnings.warn(


/var/folders/s9/rd03rq4x0px79z1jnyqlz3540000gn/T/ipykernel_10026/2233981379.py:10: PyVistaDeprecationWarning: `panel` backend is deprecated and is planned for future removal.
  pv.global_theme.jupyter_backend = "panel"


BokehModel(combine_events=True, render_bundle={'docs_json': {'449c632a-6acb-4d6d-9872-d9899d0479f7': {'version…

## SolCx from the same setup

+<br>
%%

In [21]:
stokes.bodyforce = sympy.Matrix(
    [0, -sympy.cos(sympy.pi * x) * sympy.sin(2 * sympy.pi * y)]
)

In [22]:
stokes.bodyforce[0] -= 1.0e6 * v.sym[0] * (left_fn + right_fn)
stokes.bodyforce[1] -= 1.0e6 * v.sym[1] * (surface_fn + base_fn)

In [23]:
viscosity_fn = sympy.Piecewise(
    (
        1.0e6,
        x > x_c,
    ),
    (1.0, True),
)
stokes.constitutive_model.Parameters.viscosity = viscosity_fn
stokes.saddle_preconditioner = 1 / stokes.constitutive_model.Parameters.viscosity

%%

In [24]:
stokes.constitutive_model.Parameters.viscosity
stokes.solve()

  0 SNES Function norm 0.0131676 
  1 SNES Function norm 6.38523e-05 
  2 SNES Function norm 2.22601e-07 
Nonlinear Stokes_1_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 2


3

+<br>
check the mesh if in a notebook / serial

In [25]:
import mpi4py

In [26]:
if mpi4py.MPI.COMM_WORLD.size == 1:
    import numpy as np
    import pyvista as pv
    import vtk
    pv.global_theme.background = "white"
    pv.global_theme.window_size = [750, 1200]
    pv.global_theme.antialiasing = True
    pv.global_theme.jupyter_backend = "panel"
    pv.global_theme.smooth_shading = True
    mesh.vtk("tmp_mesh.vtk")
    pvmesh = pv.read("tmp_mesh.vtk")
    pvmesh.point_data["P"] = uw.function.evaluate(p.sym[0], mesh.data)
    pvmesh.point_data["V"] = uw.function.evaluate(v.sym.dot(v.sym), mesh.data)
    arrow_loc = np.zeros((stokes.u.coords.shape[0], 3))
    arrow_loc[:, 0:2] = stokes.u.coords[...]
    arrow_length = np.zeros((stokes.u.coords.shape[0], 3))
    arrow_length[:, 0] = uw.function.evaluate(stokes.u.sym[0], stokes.u.coords)
    arrow_length[:, 1] = uw.function.evaluate(stokes.u.sym[1], stokes.u.coords)
    pl = pv.Plotter(window_size=[1000, 1000])
    pl.add_axes()
    pl.add_mesh(
        pvmesh,
        cmap="coolwarm",
        edge_color="Black",
        show_edges=True,
        scalars="V",
        use_transparency=False,
        opacity=1.0,
    )

    # pl.add_mesh(pvmesh, cmap="coolwarm", edge_color="Black", show_edges=True, scalars="T",
    #               use_transparency=False, opacity=1.0)
    pl.add_arrows(arrow_loc, arrow_length, mag=50)
    pl.show(cpos="xy")

/Users/maxwilliams/anaconda3/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2577: PyVistaDeprecationWarning: antialising is deprecated.  Please use `anti_aliasing` instead.
  warnings.warn(
/Users/maxwilliams/anaconda3/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2545: PyVistaDeprecationWarning: `anti_aliasing` is now a string or None and must be either "ssaa", "msaa", "fxaa", or None
  warnings.warn(


/var/folders/s9/rd03rq4x0px79z1jnyqlz3540000gn/T/ipykernel_10026/1657944642.py:8: PyVistaDeprecationWarning: `panel` backend is deprecated and is planned for future removal.
  pv.global_theme.jupyter_backend = "panel"


BokehModel(combine_events=True, render_bundle={'docs_json': {'776ba632-2620-4c91-bbe6-f78e48a61a4e': {'version…

%%

In [27]:
try:
    import underworld as uw2
    solC = uw2.function.analytic.SolC()
    vel_soln_analytic = solC.fn_velocity.evaluate(mesh.data)
    from mpi4py import MPI
    comm = MPI.COMM_WORLD
    from numpy import linalg as LA
    with mesh.access(v):
        num = function.evaluate(v.fn, mesh.data)  # this appears busted
        if comm.rank == 0:
            print("Diff norm a. = {}".format(LA.norm(v.data - vel_soln_analytic)))
            print("Diff norm b. = {}".format(LA.norm(num - vel_soln_analytic)))
        # if not np.allclose(v.data, vel_soln_analytic, rtol=1):
        #     raise RuntimeError("Solve did not produce expected result.")
    comm.barrier()
except ImportError:
    import warnings
    warnings.warn("Unable to test SolC results as UW2 not available.")

/var/folders/s9/rd03rq4x0px79z1jnyqlz3540000gn/T/ipykernel_10026/594938129.py:18: UserWarning: Unable to test SolC results as UW2 not available.
  warnings.warn("Unable to test SolC results as UW2 not available.")


%%

+<br>
